# AC221 Project Final Report
## Evaluating Algorithmic Fairness on the NYPD Stop, Question and Frisk Dataset

- By Phoebe and Abhi

## Background and Motivation

## Related Work

## Data


We used the dataset from NYPD Stop, Question and Frisk (SQF) Dataset from 2018. There are 11008 observations and 70% not-arrested. We define our response variable of whether a person stopped was arrested or not, which therefore, it is a binary response variable.

In [2]:
# Imports
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [3]:
sqf_df = pd.read_csv('../data/sqf-2018.csv')
sqf_df.head()

,STOP_FRISK_ID,STOP_FRISK_DATE,Stop Frisk Time,YEAR2,MONTH2,DAY2,STOP_WAS_INITIATED,RECORD_STATUS_CODE,ISSUING_OFFICER_RANK,ISSUING_OFFICER_COMMAND_CODE,SUPERVISING_OFFICER_RANK,SUPERVISING_OFFICER_COMMAND_CODE,SUPERVISING_ACTION_CORRESPONDING_ACTIVITY_LOG_ENTRY_REVIEWED,LOCATION_IN_OUT_CODE,JURISDICTION_CODE,JURISDICTION_DESCRIPTION,OBSERVED_DURATION_MINUTES,SUSPECTED_CRIME_DESCRIPTION,STOP_DURATION_MINUTES,OFFICER_EXPLAINED_STOP_FLAG,OFFICER_NOT_EXPLAINED_STOP_DESCRIPTION,OTHER_PERSON_STOPPED_FLAG,SUSPECT_ARRESTED_FLAG,SUSPECT_ARREST_OFFENSE,SUMMONS_ISSUED_FLAG,SUMMONS_OFFENSE_DESCRIPTION,OFFICER_IN_UNIFORM_FLAG,ID_CARD_IDENTIFIES_OFFICER_FLAG,SHIELD_IDENTIFIES_OFFICER_FLAG,VERBAL_IDENTIFIES_OFFICER_FLAG,FRISKED_FLAG,SEARCHED_FLAG,OTHER_CONTRABAND_FLAG,FIREARM_FLAG,KNIFE_CUTTER_FLAG,OTHER_WEAPON_FLAG,WEAPON_FOUND_FLAG,PHYSICAL_FORCE_CEW_FLAG,PHYSICAL_FORCE_DRAW_POINT_FIREARM_FLAG,PHYSICAL_FORCE_HANDCUFF_SUSPECT_FLAG,PHYSICAL_FORCE_OC_SPRAY_USED_FLAG,PHYSICAL_FORCE_OTHER_FLAG,PHYSICAL_FORCE_RESTRAINT_USED_FLAG,PHYSICAL_FORCE_VERBAL_INSTRUCTION_FLAG,PHYSICAL_FORCE_WEAPON_IMPACT_FLAG,BACKROUND_CIRCUMSTANCES_VIOLENT_CRIME_FLAG,BACKROUND_CIRCUMSTANCES_SUSPECT_KNOWN_TO_CARRY_WEAPON_FLAG,SUSPECTS_ACTIONS_CASING_FLAG,SUSPECTS_ACTIONS_CONCEALED_POSSESSION_WEAPON_FLAG,SUSPECTS_ACTIONS_DECRIPTION_FLAG,SUSPECTS_ACTIONS_DRUG_TRANSACTIONS_FLAG,SUSPECTS_ACTIONS_IDENTIFY_CRIME_PATTERN_FLAG,SUSPECTS_ACTIONS_LOOKOUT_FLAG,SUSPECTS_ACTIONS_OTHER_FLAG,SUSPECTS_ACTIONS_PROXIMITY_TO_SCENE_FLAG,SEARCH_BASIS_ADMISSION_FLAG,SEARCH_BASIS_CONSENT_FLAG,SEARCH_BASIS_HARD_OBJECT_FLAG,SEARCH_BASIS_INCIDENTAL_TO_ARREST_FLAG,SEARCH_BASIS_OTHER_FLAG,SEARCH_BASIS_OUTLINE_FLAG,DEMEANOR_CODE,DEMEANOR_OF_PERSON_STOPPED,SUSPECT_REPORTED_AGE,SUSPECT_SEX,SUSPECT_RACE_DESCRIPTION,SUSPECT_HEIGHT,SUSPECT_WEIGHT,SUSPECT_BODY_BUILD_TYPE,SUSPECT_EYE_COLOR,SUSPECT_HAIR_COLOR,SUSPECT_OTHER_DESCRIPTION,STOP_LOCATION_PRECINCT,STOP_LOCATION_SECTOR_CODE,STOP_LOCATION_APARTMENT,STOP_LOCATION_FULL_ADDRESS,STOP_LOCATION_PREMISES_NAME,STOP_LOCATION_STREET_NAME,STOP_LOCATION_X,STOP_LOCATION_Y,STOP_LOCATION_ZIP_CODE,STOP_LOCATION_PATROL_BORO_NAME,STOP_LOCATION_BORO_NAME
0,1,1/1/18,19:04:00,2018,January,Monday,Based on C/W on Scene,APP,POM,1,SGT,1,Y,(null),P,PSB,0,MENACING,18,Y,(null),Y,N,(null),N,(null),Y,(null),(null),(null),Y,N,N,(null),(null),(null),N,(null),(null),(null),(null),(null),(null),Y,(null),(null),(null),(null),Y,Y,(null),(null),(null),(null),Y,(null),(null),(null),(null),(null),(null),DE,COOPERATIVE,(null),MALE,WHITE,5.1,170,MED,BRO,BLK,(null),1,G,(null),VARICK STREET && FRANKLIN STREET,(null),VARICK STREET,982327,201274,(null),PBMS,MANHATTAN
1,2,1/1/18,23:00:00,2018,January,Monday,Based on Radio Run,APP,POM,34,SGT,34,N,O,P,PSB,1,CPW,15,Y,(null),N,N,(null),N,(null),Y,(null),(null),(null),Y,Y,N,(null),(null),(null),N,(null),Y,(null),(null),(null),(null),(null),(null),(null),(null),(null),(null),Y,(null),(null),(null),(null),(null),(null),(null),(null),(null),Y,(null),DE,NERVOUS,26,MALE,BLACK HISPANIC,6.1,250,HEA,BRO,BLK,(null),34,C,(null),DYCKMAN STREET && POST AVENUE,(null),DYCKMAN STREET,1004892,253548,(null),PBMN,MANHATTAN
2,3,1/1/18,23:55:00,2018,January,Monday,Based on Radio Run,APP,POM,808,SGT,808,Y,I,H,Housing,0,GRAND LARCENY,10,Y,(null),N,N,(null),N,(null),Y,(null),(null),(null),Y,N,N,(null),(null),(null),N,(null),(null),(null),(null),(null),(null),Y,(null),Y,(null),(null),(null),Y,(null),(null),(null),(null),Y,(null),(null),(null),(null),(null),(null),DE,RELIEVED,40,MALE,WHITE HISPANIC,5.5,150,THN,BRO,BLD,(null),43,B,4M,2245 RANDALL AVENUE,(null),RANDALL AVENUE,1026706,237776,(null),PBBX,BRONX
3,4,1/1/18,3:23:00,2018,January,Monday,Based on Radio Run,APP,POM,63,SGT,63,Y,O,P,PSB,2,ROBBERY,15,Y,(null),Y,Y,ROBBERY,N,(null),Y,(null),(null),(null),Y,Y,N,(null),(null),(null),N,(null),(null),(null),(null),(null),Y,Y,(null),Y,(null),(null),(null),Y,(null),(null),(null),Y,(null),(null),(null),(null),(null),Y,(null),DE,COMPLAIN,38,MALE,AMERICAN INDIAN/ALASKAN NATIVE,5.1,

## Data Cleaning

We preprocessed the dataset before any analysis, specifically, 
- We turned "Y" and "N" in the \*_FLAG columns to binary as we assume that those "FLAG" columns refer to binary indicators, for example, "OTHER_PERSON_STOPPED_FLAG". Some of the FLAG columns contain values that are not Y/N, which we assumed them to be Y because they could be indiciative of some form of events but in a special category. 

- We dropped irrelevant columns for modeling purpose, particuarly, we excluded 'STOP_LOCATION_PREMISES_NAME', 'STOP_FRISK_ID', 'YEAR2' and 'RECORD_STATUS_CODE'

## Exploratory Data Analysis

We looked at the distribution of stop and frisk incidents by month, race and gender. We looked at the distribution between months to see if it resembles any seasonal pattern that observed in actual crime rate. 

We compared the count between race and gender, because gender and race are defined as protected class legally. To evaluate our model fairness, particularly on the protected class, we want to know if there is any baseline difference.

Figure 1a: \# of Stopped by Month | Figure 1b: \# of Stopped and Arrested by Month
- | - 
![alt](../viz/month2.png) | ![alt](../viz/month_sep.png)

From Figure 1a, we can see that there's some variation in number of suspects stopped between months, particularly, we can note that November and December has lower numbers than others. From Figure 1b, we can see that there are some variation in both number of arrests and stopped between months.

Figure 2a: \# of Stopped by Race | Figure 2b: \# of Stopped and Arrested by Race
- | - 
![alt](../viz/race.png) | ![alt](../viz/race2.png)

From Figure 2a, we can see that Black has the highest count among the suspects stopped, however, we can see that there is a small variation of the ratio of count of arrested over count of stopped between each race, suggested by Figure 2b.

Figure 3a: \# of Stopped by Gender | Figure 3b: \# of Stopped and Arrested by Gender
- | - 
![alt](../viz/gender.png) | ![alt](../viz/gender2.png)

From Figure 3a, we can see that male has the highest count among the suspects stopped, however, we can see that the difference is a lot smaller when comparing the ratio of count of arrested over count of stopped between each gender, suggested by Figure 2b. Null indicates missing self-report gender.

Figure 4a: \# of Stopped by Age | Figure 4b: \# of Stopped and Arrested by Age
- | - 
![alt](../viz/age.png) | ![alt](../viz/age2.png)

From Figure 4a, we can see that most people who are stopped are among age 15-25, and from Figure 4b, we can see the pattern are similar between stopped only and stopped and arrested. However, we can see that proportionally, the ratio of arrest to stopped only is much higher when the suspect is older.

## Modeling

## Fairness and Interpretability Metrics
We explore a few commonly used metrics to evaluate the fairness of our model prediction and the local interpretability of our model.

### Statistical parity
Statistical parity measures the difference between the probability of positive decisions for the protected group and the probability of positive decisions for the unprotected group. Mathematically:
$$Sp = P(d=1|G=0) - P(d=1|G=1)$$

Ideally, $Sp$ should be small such that there is little difference of probability of positive decisionb etween protected and unprotected group. A large value of $Sp$ suggests potential unfairness in the dataset and $Sp$ of 0 suggests complete fairness (equal probability between classes).

In our dataset, we can examine the probability of positive decision (stopped and arrested) between different **gender** and **race**.

In the example of gender, we are comparing the proportion of positive decisions amongst male and substracting the proportion of positive decisions amongst female

$$Sp = \frac{ \text{# positive predictions and male}} { \text{ # of male} } - \frac{ \text{# positive predictions and female}} { \text{ # of female}}$$

### Conditional Parity
Besides Statistical parity, we can further evaluate if there is any difference between classes if we condition the classes on a certain condition, in other words, are we being fair, in terms of statistical parity, in that subgroup? For example, in our dataset, we can consider `race` column as our condition, and evaluate the difference between the two genders when conditioned on race. 

$$\frac{ \text{# positive predictions and black male}} { \text{ # of black male} } - \frac{ \text{# positive predictions and black female}} { \text{ # of black female}}$$

### False Positive (Negative) Error Rate Balance (FPR/FNR)
The two parity measures do not take into account the real labels of each observation; they only consider the predictions. False Positive/Negative Error Rate Balance examines if there are equal poportions of false positives/false negatives in protected and unprotected classes. This is a particularly important metric in cases where committing mistakes disproportionately for different protected groups can bring negative outcomes.

In the example of using gender as the protected class, the definitions are as following:

$FPR = \frac{ \text{# false positive in male}} { \text{ # of male} } - \frac{ \text{# false positive in female}} { \text{ # of female}}$

$FPR = \frac{ \text{# false negative in male}} { \text{ # of male} } - \frac{ \text{# false negative in female}} { \text{ # of female}}$

while false positive is defined as count of cases where the true label is false and the predicted label is true, vice versa for false negative.

### SHAP
SHAP (SHapley Additive exPlanations) is a unified approach to explain the output of any machine learning model. It assigns each feature an importance value for a particular prediction. Overall, SHAP connects game theory with local explanations, uniting several previous methods and representing the only possible consistent and locally accurate additive feature attribution method based on expectations.

#### Definition of SHAP value
Mathematically, we can break down the calculation of SHAP value as following:

$f_x(S) = f(h_x(z')) = E[f(x) | x_S]$

1. $z'$ represent a feature being observed ($z_i'$ = 1) or unknown ($z_i'$ = 0)

2. $h_x$ is a mapping that maps between a binary pattern of missing features represented by $z'$ and the original function input space.

Given such mapping, we can evaluate $f(h_x(z'))$ and so calculate the effect of observing ($z_i' = 1$) or not observing a feature ($z_i' = 0$)

3. $S$ refers the set of non-zero indexes in $z'$

4. $E[f(x) | x_S]$ is the expected value of the function conditioned on a subset $S$ of the input features.



SHAP values combine these conditional expectations with the classic Shapley values from game theory to attribute $\phi_i$ values to each feature:

$\phi_i = \sum \frac{|S|!(M-|S|-1)}{M!} [f_x(S \cup \{i\}) - f_x(S)]$

## Results

## Discussion

## Reference